# Web scraping to get data for the Mars Mission.

In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import pandas as pd 
import time
import datetime

#define exec path for chromedriver.exe.  Scrape information from https://mars.nasa.gov/news/

#for windows: 
#executable_path = {'executable_path': 'C:\ChromeSafe\chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

# # Initialize PyMongo to work with MongoDBs
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)




In [2]:
#for mac:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
#Create empty dictionaries to store all the mars information.
mars_info_dict=dict()

# Part  1.	### NASA Mars News

In [5]:

#Define url and browse the site using chrome. 
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html_1 = browser.html



In [6]:
#create soup object and use beautiful soup to parse html. 
soup1 = BeautifulSoup(html_1, 'html.parser')

In [7]:
#print(html_1)

In [8]:
#print(soup1.prettify())

In [9]:
## Mars space images 
#* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).
#* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a 
#* variable called `featured_image_url`. Make sure to find the image url to the full size `.jpg` image.
#* Make sure to save a complete url string for this image.
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)
full_image = browser.find_by_id('full_image')
full_image.click()


In [10]:
# Scrape information from html for https://mars.nasa.gov/news/. class is "content_title' for news title.
try :
    result_title = soup1.find('ul', class_="item_list").find('li',class_="slide").find('div',class_="content_title").text

    #Class is class="article_teaser_body" for para text. 
    news_body = soup1.find('ul',class_="item_list").find('li',class_="slide").find('div',class_="article_teaser_body").text

    print("The news title is " + result_title)
    #print(f"The news_title is: {news_title}") 

    print("The news body is " + news_body)
    #print(f"The News Body is: {news_body}") 

except  AttributeError as Atterror:
    print(Atterror)



The news title is For InSight, Dust Cleanings Will Yield New Science
The news body is Wind can be crucial to clearing dust from spacecraft solar panels on Mars. With InSight's meteorological sensors, scientists get their first measurements of wind and dust interacting "live" on the Martian surface.  


In [11]:
#Append results from part 1 into the final mars_info dictionary.
from pprint import pprint
mars_info_dict["Mars_news_title"] = result_title
mars_info_dict["Mars_news_body"] = news_body
pprint(mars_info_dict)

{'Mars_news_body': 'Wind can be crucial to clearing dust from spacecraft solar '
                   "panels on Mars. With InSight's meteorological sensors, "
                   'scientists get their first measurements of wind and dust '
                   'interacting "live" on the Martian surface.  ',
 'Mars_news_title': 'For InSight, Dust Cleanings Will Yield New Science'}


# Part 2.	### JPL Mars Space Images - Featured Image

In [12]:
#click on the link for "more info"
#time.sleep(15)
link_more_info = browser.find_link_by_partial_text('more info')
link_more_info.click()




In [13]:
#Retrieve  the  html from the page. Parse html using bs4 and find the path for the  full size image.
fullimg_html2 = browser.html
soup2 = BeautifulSoup(fullimg_html2, "html.parser")
fullimg_href = soup2.find('figure', class_='lede').a['href']
featured_image_url = "https://www.jpl.nasa.gov" + fullimg_href

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18840_hires.jpg


In [14]:
#Append featured image url to the Mars dictionary.
mars_info_dict["Mars_featured_image_url"] = featured_image_url

In [15]:
pprint(mars_info_dict)

{'Mars_featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18840_hires.jpg',
 'Mars_news_body': 'Wind can be crucial to clearing dust from spacecraft solar '
                   "panels on Mars. With InSight's meteorological sensors, "
                   'scientists get their first measurements of wind and dust '
                   'interacting "live" on the Martian surface.  ',
 'Mars_news_title': 'For InSight, Dust Cleanings Will Yield New Science'}


# Part 3 . ### Mars Weather tweet

In [16]:
##* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather 
##  tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.
url3 = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url3)
html3 = browser.html
soup3 = BeautifulSoup(html3, 'html.parser')


In [17]:
#print(soup.prettify())
#Save the tweet text for the weather report as a variable called `mars_weather`.
mars_weather = soup3.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather

'InSight sol 156 (2019-05-05) low -99.2ºC (-146.6ºF) high -18.1ºC (-0.6ºF)\nwinds from the SW at 4.7 m/s (10.5 mph) gusting to 13.8 m/s (30.8 mph)\npressure at 7.40 hPapic.twitter.com/FlGDeYg8hi'

In [18]:
#Add weather tweet to the mars_info dict.
mars_info_dict["Mars_tweet_weather"] = mars_weather

In [19]:
pprint(mars_info_dict)

{'Mars_featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18840_hires.jpg',
 'Mars_news_body': 'Wind can be crucial to clearing dust from spacecraft solar '
                   "panels on Mars. With InSight's meteorological sensors, "
                   'scientists get their first measurements of wind and dust '
                   'interacting "live" on the Martian surface.  ',
 'Mars_news_title': 'For InSight, Dust Cleanings Will Yield New Science',
 'Mars_tweet_weather': 'InSight sol 156 (2019-05-05) low -99.2ºC (-146.6ºF) '
                       'high -18.1ºC (-0.6ºF)\n'
                       'winds from the SW at 4.7 m/s (10.5 mph) gusting to '
                       '13.8 m/s (30.8 mph)\n'
                       'pressure at 7.40 hPapic.twitter.com/FlGDeYg8hi'}


# Part 4.	### Mars Facts

In [20]:
# Visit the Mars Facts webpage [here](http://space-facts.com/mars/) 
# and use Pandas to scrape the table containing facts 
# about the planet including Diameter, Mass, etc.
# Use Pandas to convert the data to a HTML table string
url4 = "http://space-facts.com/mars/"
df_marsfacts_all = pd.read_html(url4)
df_marsfacts = df_marsfacts_all[0]
#df_marsfacts

In [21]:
# Provide appropriate column names for the dataframe. 
df_marsfacts.columns = ['Mars_Facts', 'Values']

#convert to html
df_marsfacts.to_html("mars_facts.html", index=False)


In [22]:
#set index for better retrieval. 
df_marsfacts.set_index("Mars_Facts")
#Add another html version of the Mars facts tables.
mars_facts_html = df_marsfacts.to_html(classes="mars_facts table table-striped")
mars_info_dict["Mars_facts_table"] = mars_facts_html

In [23]:
pprint(mars_info_dict)

{'Mars_facts_table': '<table border="1" class="dataframe mars_facts table '
                     'table-striped">\n'
                     '  <thead>\n'
                     '    <tr style="text-align: right;">\n'
                     '      <th></th>\n'
                     '      <th>Mars_Facts</th>\n'
                     '      <th>Values</th>\n'
                     '    </tr>\n'
                     '  </thead>\n'
                     '  <tbody>\n'
                     '    <tr>\n'
                     '      <th>0</th>\n'
                     '      <td>Equatorial Diameter:</td>\n'
                     '      <td>6,792 km</td>\n'
                     '    </tr>\n'
                     '    <tr>\n'
                     '      <th>1</th>\n'
                     '      <td>Polar Diameter:</td>\n'
                     '      <td>6,752 km</td>\n'
                     '    </tr>\n'
                     '    <tr>\n'
                     '      <th>2</th>\n'
                     '      <

# Part 5.	### Mars Hemispheres


In [24]:
# Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars)
# to obtain high resolution images for each of Mar's hemispheres.
# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the 
# hemisphere name. 
# Use a Python dictionary to store the data using the keys `img_url` and `title`. append the dictionary with the image url 
# string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.
# use splinter and soup to retrieve html and convert to soup object. 
url5 =  "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/"
browser.visit(url5)
#time.sleep(10)
html5 = browser.html
soup5 = BeautifulSoup(html5, "html.parser")

In [25]:
#parse soup object for images of the 4 hemispheres .
class_collap_results = soup5.find('div', class_="collapsible results")
hemis_items = class_collap_results.find_all('div',class_='item')


In [ ]:
#hemis_items

In [26]:
#loop thru to find tile and the image urls to append to lists. 
hemis_img_urls_list=list()
img_urls_list = list()
title_list = list()
for h in hemis_items:
    #save title
    h_title = h.h3.text
    title_list.append(h_title)
    
    # find the href link.
    h_href  = "https://astrogeology.usgs.gov" + h.find('a',class_='itemLink product-item')['href']
    
    #print(h_title,h_href)
    
    #browse the link from each page
    browser.visit(h_href)
    time.sleep(5)
    #Retrieve the  image links and store in a list. 
    html5   = browser.html
    soup_img = BeautifulSoup(html5, 'html.parser')
    h_img_url = soup_img.find('div', class_='downloads').find('li').a['href']
    print("h_img_url" + h_img_url)
    img_urls_list.append(h_img_url)
    
    # create a dictionary with  each image and title and append to a list. 
    hemispheres_dict = dict()
    hemispheres_dict['title'] = h_title
    hemispheres_dict['img_url'] = h_img_url
    
    hemis_img_urls_list.append(hemispheres_dict)
    
print(hemis_img_urls_list)
print(title_list)
print(img_urls_list)


h_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
h_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
h_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
h_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/

In [27]:
#print(len(hemis_img_urls_list))
#Add hemispheres list  to the mars_info dictionary.
mars_info_dict["Hemisphere_image_urls"] = hemis_img_urls_list

In [28]:
pprint(mars_info_dict)

{'Hemisphere_image_urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
                            'title': 'Cerberus Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
                            'title': 'Schiaparelli Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
                            'title': 'Syrtis Major Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
                            'title': 'Valles Marineris Hemisphere Enhanced'}],
 'Mars_facts_table': '<table border="1" class="dataframe mars_facts table '
                     'table-striped">\n'
                     '  <thead>\n'

In [29]:
#Generate date time and store in the dictionary.
cur_datetime = datetime.datetime.utcnow()
mars_info_dict["Date_time"] = cur_datetime

In [30]:
pprint(mars_info_dict)

{'Date_time': datetime.datetime(2019, 5, 7, 0, 39, 14, 116584),
 'Hemisphere_image_urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
                            'title': 'Cerberus Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
                            'title': 'Schiaparelli Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
                            'title': 'Syrtis Major Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
                            'title': 'Valles Marineris Hemisphere Enhanced'}],
 'Mars_facts_table': '<table border="1" class="dataframe mars_facts table '
           